In [1]:
import os
os.system('pip install boto3')


[notice] A new release of pip is available: 23.0.1 -> 25.3
[notice] To update, run: pip install --upgrade pip


0

In [1]:
from pyspark.sql import SparkSession
from pyspark.sql.functions import *
from pyspark.sql.types import *
import boto3
import re
from botocore.client import Config
from urllib.parse import urlparse
from datetime import datetime

In [2]:
spark = SparkSession.builder \
    .appName("monitor_iceberg_data") \
    .config("spark.cores.max", "1") \
    .config("spark.executor.memory", "2g") \
    .config("spark.hadoop.fs.s3a.endpoint", "http://minio:9000") \
    .config("spark.hadoop.fs.s3a.access.key", "minioadmin") \
    .config("spark.hadoop.fs.s3a.secret.key", "minioadmin") \
    .config("spark.hadoop.fs.s3a.path.style.access", "true") \
    .config("spark.hadoop.fs.s3a.aws.credentials.provider", "org.apache.hadoop.fs.s3a.SimpleAWSCredentialsProvider") \
    .config("spark.hadoop.fs.s3a.impl", "org.apache.hadoop.fs.s3a.S3AFileSystem") \
    .config("spark.sql.extensions", "org.apache.iceberg.spark.extensions.IcebergSparkSessionExtensions") \
    .config("spark.sql.catalog.iceberg", "org.apache.iceberg.spark.SparkCatalog") \
    .config("spark.sql.catalog.iceberg.type", "hive") \
    .config("spark.sql.catalog.iceberg.uri", "thrift://hive-metastore:9083") \
    .config("spark.sql.catalog.iceberg.io-impl", "org.apache.iceberg.aws.s3.S3FileIO") \
    .config("spark.sql.catalog.iceberg.warehouse", "s3a://warehouse/") \
    .config("spark.sql.catalog.iceberg.s3.endpoint", "http://minio:9000") \
    .getOrCreate()

spark.sparkContext.setLogLevel('ERROR')

Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).
25/12/12 03:48:48 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable


In [3]:
# khai báo ngày và file ghi các table lỗi
date = datetime.now()
date = str(date.strftime('%Y-%m-%d %H:%M:%S'))

# csv_error = '/opt/airflow/spark_job/monitor_iceberg_data_table_error.csv'
catalog = "iceberg"

In [4]:
a = []
df = spark.sql(f"SHOW SCHEMAS FROM {catalog}")

for row in df.collect(): 
    namespace = row['namespace']  
    df1 = spark.sql(f"SHOW TABLES FROM {catalog}.{namespace}")
    for i in df1.collect():
        table = catalog + '.' +  i['namespace'] + '.' + i['tableName'] 
        a.append(table)

In [5]:
def convert_bytes_to_mb(size_in_bytes):
    if size_in_bytes is None:
        return None
    
    size_in_mb = size_in_bytes / (1024 * 1024)
    return size_in_mb

In [6]:
# Hàm lấy size 1 thư mục trong minio
def get_size_folder(s3a_path):
    s3 = boto3.client(
        's3',
        endpoint_url='http://minio:9000',
        aws_access_key_id='minioadmin',
        aws_secret_access_key='minioadmin',
        config=Config(signature_version='s3v4')
    )

    # Parse s3a path
    parsed_url = urlparse(s3a_path)
    bucket = parsed_url.netloc
    prefix = parsed_url.path.lstrip('/')

    total_size = 0
    total_files = 0
    continuation_token = None
   
    while True:
        if continuation_token:
            response = s3.list_objects_v2(Bucket=bucket, Prefix=prefix, ContinuationToken=continuation_token)
        else:
            response = s3.list_objects_v2(Bucket=bucket, Prefix=prefix)
            
        total_files += len(response['Contents'])
        if 'Contents' in response:
            for obj in response['Contents']:
                total_size += obj['Size']
                #total_files += 1
                
        if response.get('IsTruncated'):  # If truncated, it means there are more objects to fetch
            continuation_token = response.get('NextContinuationToken')
        else:
            break
            
    total_size = convert_bytes_to_mb(total_size)
    return total_files, total_size


In [7]:
schema = StructType([
    StructField('catalog_name', StringType(), True),
    StructField('schema_name', StringType(), True),
    StructField('table_name', StringType(), True),
    StructField('total_metadata_size', FloatType(), True),
    StructField('total_metadata_file', LongType(), True) 
])

result = spark.createDataFrame(spark.sparkContext.emptyRDD(), schema)

In [8]:
for table_old in a:
    try:
        # lấy metadata
        print(f"Đang xử lí {table_old}")
        sc = table_old.split('.')[1]
        tb = table_old.split('.')[2]
        
         # Lấy lệnh tạo bảng
        location = spark.sql(f"SHOW CREATE TABLE {table_old};").tail(1)[0]["createtab_stmt"]
        # Trích xuất location từ metadata
        match = re.search(r"LOCATION\s+'([^']*)'", location)
        if match:
            location = match.group(1) + '/metadata'
        # Tính toán dung lượng và số lượng file metadata
        total_metadata_file, total_metadata_size = get_size_folder(location)
        # Tạo DataFrame chứa kết quả
        data = [Row(catalog_name=catalog, schema_name=sc, table_name=tb, total_metadata_size=total_metadata_size, total_metadata_file=total_metadata_file)]
        new_df = spark.createDataFrame(data, schema)
        
        result = result.union(new_df)
    except Exception as e:
        print(f"Đã có lỗi xảy ra khi xử lý {table_old}: {e}")

Đang xử lí iceberg.bronze.amazon_products
Đang xử lí iceberg.monitor.monitor_iceberg_data
Đang xử lí iceberg.silver.amazon_products_clean
Đang xử lí iceberg.test.test_iceberg_v1
Đang xử lí iceberg.test.test_iceberg
Đang xử lí iceberg.test.data_demo
Đang xử lí iceberg.test.test_spark_kafka
Đang xử lí iceberg.test.test_pyiceberg


In [9]:
result = result.withColumn("total_metadata_size", round(result["total_metadata_size"], 3).cast('string'))
result = result.withColumn("total_metadata_size", round(result["total_metadata_size"], 3))

In [10]:
result = result.withColumn('ngay_cap_nhat', current_timestamp())
# result = result.coalesce(10)

In [11]:
result.show()

[Stage 3:===========================================>            (55 + 15) / 71]

+------------+-----------+--------------------+-------------------+-------------------+--------------------+
|catalog_name|schema_name|          table_name|total_metadata_size|total_metadata_file|       ngay_cap_nhat|
+------------+-----------+--------------------+-------------------+-------------------+--------------------+
|     iceberg|     bronze|     amazon_products|              0.025|                  4|2025-12-12 03:49:...|
|     iceberg|    monitor|monitor_iceberg_data|              0.015|                  4|2025-12-12 03:49:...|
|     iceberg|     silver|amazon_products_c...|              0.027|                  4|2025-12-12 03:49:...|
|     iceberg|       test|     test_iceberg_v1|               0.02|                  6|2025-12-12 03:49:...|
|     iceberg|       test|        test_iceberg|              0.038|                  8|2025-12-12 03:49:...|
|     iceberg|       test|           data_demo|              0.016|                  4|2025-12-12 03:49:...|
|     iceberg|     

In [12]:
result.printSchema()

root
 |-- catalog_name: string (nullable = true)
 |-- schema_name: string (nullable = true)
 |-- table_name: string (nullable = true)
 |-- total_metadata_size: double (nullable = true)
 |-- total_metadata_file: long (nullable = true)
 |-- ngay_cap_nhat: timestamp (nullable = false)



In [16]:
table = f'{catalog}.monitor.monitor_iceberg_metadata'
result.write \
  .format("iceberg") \
  .mode("overwrite") \
  .saveAsTable(table) 

In [17]:
df = spark.read \
    .format('iceberg') \
    .load('iceberg.monitor.monitor_iceberg_metadata')
df.show()

+------------+-----------+--------------------+-------------------+-------------------+--------------------+
|catalog_name|schema_name|          table_name|total_metadata_size|total_metadata_file|       ngay_cap_nhat|
+------------+-----------+--------------------+-------------------+-------------------+--------------------+
|     iceberg|     bronze|     amazon_products|              0.025|                  4|2025-12-12 03:50:...|
|     iceberg|    monitor|monitor_iceberg_data|              0.015|                  4|2025-12-12 03:50:...|
|     iceberg|     silver|amazon_products_c...|              0.027|                  4|2025-12-12 03:50:...|
|     iceberg|       test|     test_iceberg_v1|               0.02|                  6|2025-12-12 03:50:...|
|     iceberg|       test|        test_iceberg|              0.038|                  8|2025-12-12 03:50:...|
|     iceberg|       test|           data_demo|              0.016|                  4|2025-12-12 03:50:...|
|     iceberg|     

In [18]:
spark.stop()